In [1]:
import cv2
import numpy as np
import skimage
from skimage.exposure import equalize_hist
import matplotlib.pyplot as plt
import pandas as pd
from math import sqrt
from glob import glob
from scipy.spatial.distance import cdist
import IPython.display as ipd
from tqdm import tqdm

import subprocess

# Helper Functions 

In [2]:
def get_image_boxes(outputs, image_width, image_height, classes, confidence_threshold=0.4, nms_threshold=0.3):
    class_ids = []
    confidences = []
    boxes = []

    for output in outputs:
        for detection in output:
            scores = detection[5]
            class_id = np.argmax(scores)
            class_name = classes[class_id]
            confidence = scores[class_id]
            if confidence > confidence_threshold and class_name== 'Robot1':
                cx, cy, width, height = (detection[0:4] * np.array([image_width, image_height, image_width, image_height])).astype("int")
                x = int(cx - width / 2)
                y = int(cy - height / 2)
                boxes.append([x, y, int(width), int(height),cx,cy])
                confidences.append(float(confidence))
    nms_indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, nms_threshold)
    
    return [boxes[ind] for ind in nms_indices.flatten()]
def compute_point_perspective_transformation(matrix,boxes):
    list_downoids = [[box[4], box[5]+box[3]//2] for box in boxes]
    list_points_to_detect = np.float32(list_downoids).reshape(-1, 1, 2)
    transformed_points = cv2.perspectiveTransform(list_points_to_detect, matrix)
    transformed_points_list = list()
    for i in range(0,transformed_points.shape[0]):
        transformed_points_list.append([transformed_points[i][0][0],transformed_points[i][0][1]])
    return np.array(transformed_points_list).astype('int')



def eucledian_distance(point1, point2):
    x1,y1 = point1
    x2,y2 = point2
    return sqrt((x1-x2)**2 + (y1-y2)**2)

def get_birds_eye_view_image(green_box, red_box,eye_view_height,eye_view_width):
    blank_image = cv2.imread('black_background.png')
    
    cv2.putText(blank_image, str(len(red_box)), (120,100), cv2.FONT_HERSHEY_SIMPLEX , 2, (0,0,255), 4, cv2.LINE_AA) 
    cv2.putText(blank_image, str(len(green_box)), (520,100), cv2.FONT_HERSHEY_SIMPLEX , 2, (0,255,0), 4, cv2.LINE_AA)
    
    for point in green_box:
        cv2.circle(blank_image,tuple([point[6],point[7]]),20,(0,255,0),-1)
    for point in red_box:
        cv2.circle(blank_image,tuple([point[6],point[7]]),20,(0,0,255),-1)
    blank_image = cv2.resize(blank_image,(eye_view_width,eye_view_height))
    return blank_image


def get_box_image(new_box_image,box):
    for point in box:
        cv2.rectangle(new_box_image,(point[0],point[1]),(point[0]+point[2],point[1]+point[3]),(0, 255, 0), 2)
    return new_box_image
def get_boxes(birds_eye_points,boxes):
    boxes = []
    new_boxes = [tuple(box) + tuple(result) for box, result in zip(boxes, birds_eye_points)]
    for i in range(0, len(new_boxes)-1):
            for j in range(i+1, len(new_boxes)):
                cxi,cyi = new_boxes[i][6:]
                cxj,cyj = new_boxes[j][6:]
                boxes.append(new_boxes[i])
                boxes.append(new_boxes[j])

    boxes = list(set(new_boxes))
    return (boxes)

def transform_position(pos,H_matrix):
    list_points_to_detect = np.float32(pos).reshape(-1, 1, 2)
    transformed_points = cv2.perspectiveTransform(list_points_to_detect, H_matrix)
    return np.asarray(transformed_points).reshape(-1)

def eucledian_distance(point1, point2):
    x1,y1 = point1
    x2,y2 = point2
    return sqrt((x1-x2)**2 + (y1-y2)**2)

# Points Choose to transform

In [3]:
# Define the ROI and corresponding points for IPM
ipm_pts = np.array([[0, 0],  [150,155], [660, 155], [405, 605]], dtype=np.float32)
roi = np.array([[349, 133], [386, 236], [1220, 124], [282, 914]], dtype=np.float32)

img = cv2.imread('R3_Moment.jpg')
# Compute the IPM matrix
H_matrix = cv2.getPerspectiveTransform(roi, ipm_pts)
dst_size=(1080, 1920)
# Apply the IPM matrix to the input image
ipm = cv2.warpPerspective(img, H_matrix, dst_size)

# Display the input image and the IPM result
cv2.imshow('Input Image', img)
cv2.imshow('IPM Result', ipm)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Import ML Model and Predict

In [4]:
from roboflow import Roboflow
rf = Roboflow(api_key="0HPgx6Yl0BWMjeGp9Wsc")
project = rf.workspace().project("autoref")
model = project.version(1).model

loading Roboflow workspace...
loading Roboflow project...


# Model Predictions

In [5]:
####FARAH'S and AHMET's version with ROBOFLOW prediction
# Open the video file for reading

cap = cv2.VideoCapture("R2.mp4")
n_frames=int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# Get the video frame rate and dimensions
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create a VideoWriter object for writing the output video
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter("output.mp4", fourcc, fps, (width, height))

color_list = [(0, 255, 255), (255, 0, 0),(0, 125, 255),(0, 0, 255) ]

for frame in range(n_frames):
    # read the current frame
    ret, img =cap.read()
    if ret == False:
        break

    # predict on the current frame using the model
    response = model.predict(img, confidence=40 , overlap=50)
    # Draw predicted boxes on the frame
    dtc = []
    for detection in response.json()["predictions"]:
        if detection["class"] == 'Ball' :
            detection["class"] = 0
        elif detection["class"] == "Team_Blue" :
            detection["class"] = 1
        else:
            detection["class"] = 2
        temp = [detection["x"],
                    detection["y"],
                    detection["width"],
                    detection["height"],
                    detection["confidence"],
                    detection["class"]]

        dtc.append(temp)
    del temp
    
    dtc = np.array(dtc)
    L = dtc.shape[0]
    temp_H = [] #create a temporary array to store H_transformation results

    for i in range(len(dtc)): #for every object, transform coordinates
        temp_H.append(transform_position(dtc[i][:2], H_matrix))
    temp_H = np.array(temp_H)

    dtc = np.append(dtc, temp_H, axis = 1) #append transformed coordinates in to results array for each obj

    dtc = dtc[dtc[:,5].argsort()] #sort obj wrt the class id - first one is always ball if there is one

    pos_check = dtc[:, 6:] - dtc[0, 6:] #check every robots pos wrt ball position
    dtc = np.append(dtc, pos_check, axis =1) #append relative positions in to array

    box_pos = []
    box_pos.append([dtc[:,0] - dtc[:,2]/2, dtc[:,1] - dtc[:,3]/2, dtc[:,0] + dtc[:,2]/2, dtc[:,1] + dtc[:,3]/2])
    box_pos = np.array(box_pos).T.reshape(L,4)

    dtc = np.concatenate((dtc, box_pos), axis = 1)
    del box_pos
   

    if dtc[0][5] == 0: #if the ball is detected
        
        
        #put a box for ball with proper color
        x1 = dtc[0][-4]
        y1 = dtc[0][-3]
        x2 = dtc[0][-2]
        y2 = dtc[0][-1]
        cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), color_list[0], 2)

        
        
        
        for i in range(1,len(dtc)): 
            
            #box dimensions for every obj detected
            x1 = dtc[i][-4]
            y1 = dtc[i][-3]
            x2 = dtc[i][-2]
            y2 = dtc[i][-1]

            #distance of the robots with respect to ball
            dist = eucledian_distance(dtc[i][6:8],dtc[0][6:8])
            


            
            #dtc[i][6/7] is pposition rel. to ball dtc[i][5] is class id            
            if dtc[i][6] < 0 and dtc[i][7] < 0 and dtc[i][5] != 0: #if a robot is behind the ball
                corner = dtc[i] #the robot to kick corner
                idx_corner = i #index of the robot to kick corner
            
            
            #Procedure Check
            
            #Same team players are allowed to 2 meters
            if (dtc[i][5] == corner[5]):
                if idx_corner != i: 
                    if dist < 200: 
                        cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), color_list[3], 2)
                        cv2.putText(img,"There is a violation by attacking team", (int(x1), int(y2) + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_list[3], 2)
                        cv2.putText(img,"{} {:.2f}".format("Distance to ball", dist/100), (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_list[3], 2)

                    else:
                        cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), color_list[int(corner[5])], 2)
                        cv2.putText(img,"{} {:.2f}".format("Distance to ball", dist/100), (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_list[int(corner[5])], 2)

                else:
                    cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), color_list[int(corner[5])], 2)

                    
                
            #Opposing team are allowed to 3meters
            else:
                cv2.putText(img,"{} {:.2f}".format("Distance to ball", dist/100), (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_list[int(dtc[i][5])], 2)
                if dist < 300 :
                    cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), color_list[3], 2)
                    cv2.putText(img,"There is a violation by defending team", (int(x1), int(y2) + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_list[3], 2)
                else:
                    cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), color_list[int(dtc[i][5])], 2)


        
    # Write the frame to the output video
    out.write(img)

    # Display the frame (optional)
    cv2.imshow("frame", img)
    if cv2.waitKey(1) == ord("q"):
        break

# Release video capture and video writer
cap.release()
out.release()

# Close all windows
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [ ]:
i